## Noise Equivelant Temperture Difference

In [ ]:
# NETD — Photon Units (Jupyter-ready, sci-notation prints)

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, Layout, widgets, FloatText

# ---- Constants ----
c  = 2.99792458e8     # m/s
c2 = 1.438776877e4    # µm·K
h  = 6.62607015e-34   # J·s
q  = 1.602176634e-19  # C

# legacy constants
c1w = 3.7418e4        # W·µm^4/cm^2
c1p = 5.9958e10       # µm^4/(cm^2·s)

def plot_func(Temp,wvl1,wvl2,Tran,FNO,QE,Tint,Pitch,DarkC,RFNe):

    if wvl1 <= 0.5: wvl1 = 0.5
    if wvl2 >= 15:  wvl2 = 15

    step_um = 0.10
    wvl_set = np.arange(wvl1, wvl2, step_um)

    Mp = (1.88e27/(wvl_set**4))*(1.0/(np.exp(14388.0/(wvl_set*Temp)) - 1.0))/1.0e4     # photons/(cm^2·s·µm)
    Mw = 37400.0/(wvl_set**5)*(1.0/(np.exp(14400.0/(wvl_set*Temp)) - 1.0))              # W/(cm^2·µm)

    Mp_Tot = np.sum(Mp)*step_um
    Mw_Tot = np.sum(Mw)*step_um

    Lp = Mp_Tot/np.pi
    Lw = Mw_Tot/np.pi
    print(f"Radiance (energy)  = {Lw:.3e} W/(cm^2·sr)")
    print(f"Radiance (photon)  = {Lp:.3e} photons/(cm^2·s·sr)")

    D = 1.0
    f = FNO*D #could have use FNO below.
    FluxW = (Tran*Lw*np.pi*(Pitch*1e-4)**2*(D*0.01)**2) / (4.0*(f*0.01)**2)
    FluxP = (Tran*Lp*np.pi*(Pitch*1e-4)**2*(D*0.01)**2) / (4.0*(f*0.01)**2)
    print(f"FluxW              = {FluxW:.3e} W")
    print(f"FluxP              = {FluxP:.3e} photons/s")
 
    PhotoElec = FluxP*QE*Tint
    print(f"PhotoElec          = {PhotoElec:.3e} electrons in well")
    
    CalcResults = widgets.HTML(value = "<p style='font-size:14pt'><b>Calculated Results:</b>")
    display(CalcResults) #Displays calculated results message
   
    # ---- Planck derivative in photon units ----
    wvlm_um = 0.5*(wvl1 + wvl2)
    wvl_peak = 3669/Temp
    if wvl2 <= wvl_peak:
        shift = 1 + .075*((wvl_peak - wvl2)/7) #shift the median wvl of the plank curve to account for more photons towards lonfer wvl
        wvlm_um = wvlm_um*shift
    if wvlm_um >= wvl2:
        wvlm_um = wvl2
    lam_m   = wvlm_um*1e-6
    t1      = np.exp(c2/(wvlm_um*Temp))
    bose    = 1.0/(t1 - 1.0)

    Lpd_ph  = (2.0*c)/(lam_m**4) * bose * 1e-10
    PLDR2   = Lpd_ph * (c2/(wvlm_um*Temp**2)) * (t1/(t1 - 1.0))
    #print(f"L_p(λm)            = {Lpd_ph:.3e} photons/(cm^2·s·sr·µm)")
    #print(f"dL_p/dT            = {PLDR2:.3e} photons/(cm^2·s·sr·µm·K)")

    # ---- Noise terms ----
    NPe     = PhotoElec
    NPhoton = NPe/QE
    PNe     = np.sqrt(NPhoton)
    DNe     = np.sqrt((DarkC*Tint)/q)
    TNe     = np.sqrt(DNe**2 + PNe**2 + RFNe**2)
    NEC     = TNe
    NEI     = NEC/(QE*Tint*(Pitch*1e-4)**2)
    SNR     = NPe/NEC
    print('Noise: Dark e =  ',int(DNe),' , Photon e = ',int(PNe), ' , Roic e =',int(RFNe) )
    print(f"NEC                = {int(NEC)} e-")
    print(f"SNR                = {SNR:.3f}")
    print(f"NEI                = {NEI:.3e} photons/(s·cm^2)")

    NEP   = NEI*(h*c/lam_m)*((Pitch*1e-4)**2)
    NEB   = 1.0/(2.0*Tint)
    DSTAR = ((Pitch*1e-4)*np.sqrt(NEB))/NEP
    print(f"D*                 = {DSTAR:.3e} cm·Hz^1/2/W")
    print(f"NEP                = {NEP:.3e} W")

    E_ph = h*c/lam_m                          # [J/photon] at center λ
    DSTAR_ph = DSTAR * E_ph                   # "photon Jones": [cm·Hz^1/2 / (photons/s)]
    
    num = (FNO**2)*np.sqrt(1.0/(2.0*Tint))
    den = Tran*Pitch*1e-4*DSTAR_ph*PLDR2
    NETD = (4.0*num)/(np.pi*den)
    netd_mK = 1e3*NETD
    print(f"NETD               = {NETD:.3e} K  ({netd_mK:.3f} mK)")

    display(widgets.HTML(
        value=f"<b><p style='font-size:10pt'><b>NETD = {netd_mK:.3f} mK</b></p>"
    ))
    
    # ---- Plots ----
    plt.style.use('classic')
    plt.figure(figsize=(12,7))
    numpts   = 20
    templow  = 0.9*Temp
    temphigh = 1.1*Temp
    stepT    = (temphigh - templow)/numpts
    Templist = np.arange(templow, temphigh, stepT)
    t2R_ph   = (2.0*c)/(lam_m**4) * 1e-10

    NETDlist = []
    for Tcur in Templist:
        t1c   = np.exp(c2/(wvlm_um*Tcur))
        bosec = 1.0/(t1c - 1.0)
        Lcur  = t2R_ph * bosec
        dLdT  = Lcur * (c2/(wvlm_um*Tcur**2)) * (t1c/(t1c - 1.0))
        den_c = Tran*Pitch*1e-4*DSTAR_ph*dLdT
        NETDlist.append((4.0*num)/(np.pi*den_c))

    plt.plot(Templist, NETDlist, 'b', marker='*', linestyle='-')
    plt.title('Noise Equivalent Temperature Difference')
    plt.grid(True)
    plt.xlabel('Temperature (K)')
    plt.ylabel('NETD (K)')
    plt.axis([Templist.min(), Templist.max(), 0.5*min(NETDlist), 1.1*max(NETDlist)])

    # --- Crosshairs at (Temp, NETD) ---
    ax = plt.gca()
    ax.axvline(Temp, linestyle=':', linewidth=2)
    ax.axhline(NETD, linestyle=':', linewidth=2)
    ax.plot([Temp], [NETD], marker='o', markersize=6)

    # Annotate values (sci/engineering-friendly)
    ax.annotate(f"{Temp:.1f} K",
            xy=(Temp, ax.get_ylim()[0]),
            xytext=(6, 6), textcoords='offset points')

    ax.annotate(f"{1e3*NETD:.3f} mK",
            xy=(ax.get_xlim()[0], NETD),
            xytext=(6, 6), textcoords='offset points')

    
    #NEI plot
    #Create plots 
    numpts = 20
    templow = Temp*0.9 #sweep temp for NETD +/- 10%
    temphigh = Temp*1.1
    step = (temphigh - templow)/numpts
    Templist = np.arange(templow,temphigh,step)
    Tintlow = Tint*0.001   #sweep Tint for NEI  4 OOM use log plot
    Tinthigh = Tint*1000
    Tintstep = (Tinthigh - Tintlow)/numpts
    Tintlist = np.geomspace(Tintlow,Tinthigh,numpts)
    NEIlist = []
    
    #NIE List calculation loop
    cnt=0
    for i in Templist:
        #t1list.append(np.exp(c2/(wvl*Templist[cnt])))
        #t2= c1/wvl**5
        #t3list.append((1/(t1list[cnt] - 1)))
        #Llist.append(t3list[cnt]*t2/np.pi)
        #PLDRlist.append((Llist[cnt]*c2*t1list[cnt])/(wvl*(Templist[cnt]**2)*(t1list[cnt] - 1)))
        #denlist.append(Tran*Pitch*0.0001*DSTAR*PLDRlist[cnt])
        #NETDlist.append((4*num)/(np.pi*denlist[cnt]))
        NEIlist.append(NEC/(QE*Tintlist[cnt]*(Pitch*1e-4)**2))
        cnt+=1
        
    #def slider_func(Tintl):
    #Tints=np.exp(Tintl)
    Tints=np.exp(np.log(21*Tint))
    #print ("Slider Integration time =","{:.3e}".format(Tints) + " Seconds")
    plt.style.use('dark_background')
    plt.figure(figsize=(12,7))
    plt.plot(Tintlist,NEIlist,'r')
    plt.plot(Tintlist,NEIlist,'*w')
    plt.title('Noise Equivalent Irradiance vs Integration Time')
    plt.axis([Tintlow,Tinthigh,0.5*min(NEIlist),1.1*max(NEIlist)])
    plt.yscale('log')
    plt.xscale('log')
    #plt.xticks(np.arange(templow, temphigh, 4*step))
    plt.grid(True)
    plt.xlabel('Integration Time (Sec)')
    plt.ylabel('NEI (Photons/sec-cm^2)')
    #plt.axis([templow,temphigh-step,0.5*min(NETDlist),1.1*max(NETDlist)])
    
    #make a horiz line
    hliney = np.repeat(NEI, 2)
    hlinex = [Tintlow/2,Tint]
    plt.plot (hlinex,hliney,'w:', linewidth = 1.5)
    tNEI="{:.3e}".format(NEI)
    plt.text(Tintlow, 1.1*NEI,"  " + str(tNEI) ,color='White')
    plt.text(1.04*Tint, 0.6*min(NEIlist), str(Tint) ,color='White')
    plt.text(0.94*Tint, 0.88*NEI,"*",color='White')
        
    #make a vertical line at baseline and slider value Tint 
    NEIs=NEC/(QE*Tints*(Pitch*1e-4)**2)
    tNEIs="{:.3e}".format(NEIs)
    tTints="{:.3e}".format(Tints)
    vlinex2 = np.repeat(Tint, 2)
    vliney2 = [0,NEI]
    plt.plot (vlinex2,vliney2,'w:', linewidth = 2.0)
    vlinex3 = np.repeat(Tints, 3)
    vliney3 = [0,1000*NEI,10000*NEI]
    #plt.plot (vlinex3,vliney3,'m:', linewidth = 2.0)
    #plt.text(Tints, 1.08*NEIs,"  " + str(tNEIs) ,color='Magenta')
    #plt.text(1.04*Tints, 0.6*min(NEIlist), str(tTints) ,color='Magenta')
    #plt.text(1.04*min(Tintlist), 0.96*NEIs, "<" ,color='Magenta')
    #plt.text(0.94*Tints, 0.88*NEIs,"*",color='Magenta')
    
    #interact(slider_func, Tintl = widgets.FloatSlider(value=np.log(21*Tint), min=np.log(min(Tintlist)), max=np.log(max(Tintlist)),description = ' ln(Tint)',layout=Layout(width="900px"),continuous_update=False))
    
    
    
    
style1 = {'description_width': 'initial'}
_ = interact(
    plot_func,
    Temp  = FloatText(value=300,  description='Temperature (K)',        style=style1),
    wvl1  = FloatText(value=3.0,  description='Wavelength1 (µm)',       style=style1),
    wvl2  = FloatText(value=5.0,  description='Wavelength2 (µm)',       style=style1),
    Tran  = FloatText(value=0.8,  description='Transmission',           style=style1),
    FNO   = FloatText(value=2.0,  description='Focal Ratio (f/#)',      style=style1),
    QE    = FloatText(value=0.8,  description='Quantum Efficiency',     style=style1),
    Tint  = FloatText(value=3e-3, description='Tint (s)',               style=style1),
    Pitch = FloatText(value=20,   description='Pixel Pitch (µm)',       style=style1),
    DarkC = FloatText(value=3e-12, description='Dark Current (A)',      style=style1),
    RFNe  = FloatText(value=600,  description='ROIC Fixed Noise (e-)',  style=style1),
)
